In [1]:
import seaborn as sns
from IPython.display import Image
import plotly
import matplotlib as plt
import pandas as pd
import numpy as np
import statsmodels as sm
import copy
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import time

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
#Image("scheme.png")

In [5]:
data = pd.read_csv('/home/siarhei/Programming/ML/internship/data_after_eda.csv')
sempling = pd.read_csv('/home/siarhei/Programming/ML/internship/elapsed/final_df.csv') #after sempling(10%)

In [6]:
data = pd.concat([sempling,data[data.date_block_num>30]])

In [7]:
# load data
train=data[data.is_train == 1]
test=data[data.is_train == 0]

In [8]:
del sempling
del data

In [ ]:
import itertools
import random
import time

#worked version, just calculated from PC
def sempling(matrix, num_of_new_row):
    start_time = time.time()
    filtered_df = matrix[(matrix['shop_id'].isin(matrix['shop_id'].unique())) & (matrix['item_id'].isin(matrix['item_id'].unique()))]
    grouped = filtered_df.groupby(['shop_id','item_id'])['date_block_num'].min()
    all_date_block_nums = matrix.date_block_num.unique()
    for shop_id, item_id in itertools.product(matrix['shop_id'].unique(), matrix['item_id'].unique()):
        try: 
            first_date_block_num = grouped[shop_id, item_id] 
            block_num = np.random.choice(np.extract(all_date_block_nums > first_date_block_num, all_date_block_nums))
        except: continue
        if num_of_new_row == 0 :
            return matrix, grouped
        latest_row = filtered_df[(filtered_df['shop_id'] == shop_id) & (filtered_df['item_id'] == item_id) & (filtered_df['date_block_num'] < block_num)].sort_values(by='date_block_num', ascending=False).tail(1)
        new_row = {'date_block_num': block_num,
        'shop_id': shop_id,
        'item_id': item_id,
        'item_price': latest_row['item_price'].values[0],
        'item_cnt_day': 0.0,
        'is_train': latest_row['is_train'].values[0],
        'city_code': latest_row['city_code'].values[0],
        'shop_type_code': latest_row['shop_type_code'].values[0],
        'item_category_id': latest_row['item_category_id'].values[0],
        'platform_id': latest_row['platform_id'].values[0],
        'supercategory_id': latest_row['supercategory_id'].values[0],
        'item_name_group': latest_row['item_name_group'].values[0],
        'monthly_sales': 0.0}
        matrix = matrix.append(new_row, ignore_index=True)
        num_of_new_row = num_of_new_row - 1
        end_time = time.time()
        elapsed_time = end_time - start_time
    return matrix, grouped

start_time = time.time()
df_for_task = copy.deepcopy(b[b.date_block_num<=30])
final_df, ba = sempling(df_for_task, 350000)
end_time = time.time()

elapsed_time = end_time - start_time
print("Elapsed time : ", elapsed_time)

In [9]:
class Feature_extraction:
    def __init__(self, train, test):
        self.train = train
        self.test = test
        self.matrix = []
    
    def preprocess(self):
        from itertools import product
        cols  = ["date_block_num", "shop_id", "item_id"]
        for i in range(34):
            sales = self.train[self.train.date_block_num == i]
            self.matrix.append( np.array(list( product( [i], sales.shop_id.unique(), sales.item_id.unique() ) ), dtype = np.int16) )
        
        self.matrix = pd.DataFrame( np.vstack(self.matrix), columns = cols )
        self.matrix["date_block_num"] = self.matrix["date_block_num"].astype(np.int8)
        self.matrix["shop_id"] = self.matrix["shop_id"].astype(np.int8)
        self.matrix["item_id"] = self.matrix["item_id"].astype(np.int16)
        self.matrix.sort_values( cols, inplace = True )
    
    def add_revenue(self, cols):
        self.train["revenue"] = self.train["item_cnt_day"] * self.train["item_price"]
        group = self.train.groupby( ["date_block_num", "shop_id", "item_id"] ).agg( {"item_cnt_day": ["sum"]} )
        group.columns = ["item_cnt_month"]
        group.reset_index( inplace = True)
        self.matrix = pd.merge( self.matrix, group, on = cols, how = "left" )
        self.matrix["item_cnt_month"] = self.matrix["item_cnt_month"].fillna(0).astype(np.float16)
    
    def reduce_test_memory(self):
        self.test["date_block_num"] = self.test["date_block_num"].astype(np.int8)
        self.test["shop_id"] = self.test.shop_id.astype(np.int8)
        self.test["item_id"] = self.test.item_id.astype(np.int16)
    
    def concatenate_to_mx(self, cols):
        self.matrix = pd.concat([self.matrix, self.test.drop(["ID"],axis = 1)], ignore_index=True, sort=False, keys=cols)
        self.matrix.fillna( 0, inplace = True )
    
    def lag_feature(self,lags, cols):
        for col in cols:
            print(col)
            tmp = self.matrix[["date_block_num", "shop_id","item_id",col]]
            for i in lags:
                shifted = tmp.copy()
                shifted.columns = ["date_block_num", "shop_id", "item_id", col + "_lag_"+str(i)]
                shifted.date_block_num = shifted.date_block_num + i
                self.matrix = pd.merge(self.matrix, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    
    def add_global_item_age(self):  
        oldest_date_year_month = self.matrix.groupby('item_id')['date_block_num'].min()
        merged_df = pd.merge(self.matrix, oldest_date_year_month, on='item_id')
        merged_df = merged_df.rename(columns={'date_block_num_y': 'oldest_date'})
        merged_df['item_age'] = merged_df['date_block_num_x'] - merged_df['oldest_date']
        merged_df.drop(columns=['oldest_date'], inplace=True)
        self.matrix = copy.deepcopy(merged_df)
        self.matrix.rename(columns={'date_block_num_x':'date_block_num'}, inplace=True)
    
    def add_avg_sales(self, nan_values=0.0):
        self.matrix['average_prev_sales'] = np.nan
        date_block_nums = self.matrix['date_block_num'].unique()

        for date_block_num in date_block_nums:
            if date_block_num == 0:
                prev_sales = self.matrix[(self.matrix['date_block_num'] == date_block_num)&(self.matrix['item_cnt_month']!=0.0)]
                prev_sales = prev_sales.groupby('item_id')['item_cnt_day'].sum()/(date_block_num+1)
                self.matrix.loc[self.matrix['date_block_num'] == date_block_num,'average_prev_sales'] = nan_values
                continue
            prev_sales = self.matrix[(self.matrix['date_block_num'] < date_block_num)&(self.matrix['item_cnt_month']!=0.0)]
            prev_sales = prev_sales.groupby('item_id')['item_cnt_day'].sum()/(date_block_num)
            self.matrix.loc[self.matrix['date_block_num'] == date_block_num,'average_prev_sales'] = self.matrix.loc[self.matrix['date_block_num'] == date_block_num,'item_id'].map(prev_sales)
        self.matrix.average_prev_sales.fillna(nan_values, inplace=True)
    
    def add_shop_age(self):
        min_date_block_num = self.matrix.groupby('shop_id')['date_block_num'].min()
        self.matrix = pd.merge(self.matrix, min_date_block_num, on='shop_id', how='left', suffixes=('', '_min'))
        self.matrix['shop_age_in_months'] = self.matrix['date_block_num'] - self.matrix['date_block_num_min']
        self.matrix.drop(columns=['date_block_num_min'], inplace=True)
        
    def add_city_codes(self, shops):
        #name fix
        shops.loc[
            shops.shop_name == 'Сергиев Посад ТЦ "7Я"', "shop_name"
        ] = 'СергиевПосад ТЦ "7Я"'
        
        
        shops["city"] = shops["shop_name"].str.split(" ").map(lambda x: x[0])
        shops.loc[shops.city == "!Якутск", "city"] = "Якутск"
        shops["city_code"] = shops["city"].factorize()[0]
        shop_labels = shops[["shop_id", "city_code"]]
        self.matrix = self.matrix.merge(shop_labels, on='shop_id', how='left')
    
    def add_shop_type(self, shops):    
        #add shop_type as new column (str) *can better
        shops['shop_type'] = shops.apply(lambda row: 'ТЦ' if row['shop_name'].find('ТЦ')>-1\
                                else 'ТРЦ' if row['shop_name'].find('ТРЦ')>-1\
                                else 'ТРК' if row['shop_name'].find('ТРК')>-1\
                                else 'МТРЦ' if row['shop_name'].find('МТРЦ')>-1\
                                else 'ТК' if row['shop_name'].find('ТК')>-1\
                                else 'Интернет' if row['shop_name'].find('Интернет')>-1\
                                else 'Склад' if row['shop_name'].find('склад')>-1\
                                else 'Магаз', axis =1)
    
        #str -> int (mb int 16 and etc)
        shops["shop_type_code"] = shops["shop_type"].factorize()[0]
        shop_labels = shops[["shop_id", "shop_type_code"]]
        self.matrix = self.matrix.merge(shop_labels, on='shop_id', how='left')

    def add_cats(self,items):
        #from First Place Solution Kaggle Predict Future Sales
        self.matrix = self.matrix.merge(items[['item_id', 'item_category_id']], on='item_id', how='left')
        platform_map = {
            0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 8, 10: 1, 11: 2,
            12: 3, 13: 4, 14: 5, 15: 6, 16: 7, 17: 8, 18: 1, 19: 2, 20: 3, 21: 4, 22: 5,
            23: 6, 24: 7, 25: 8, 26: 9, 27: 10, 28: 0, 29: 0, 30: 0, 31: 0, 32: 8, 33: 11,
            34: 11, 35: 3, 36: 0, 37: 12, 38: 12, 39: 12, 40: 13, 41: 13, 42: 14, 43: 15,
            44: 15, 45: 15, 46: 14, 47: 14, 48: 14, 49: 14, 50: 14, 51: 14, 52: 14, 53: 14,
            54: 8, 55: 16, 56: 16, 57: 17, 58: 18, 59: 13, 60: 16, 61: 8, 62: 8, 63: 8, 64: 8,
            65: 8, 66: 8, 67: 8, 68: 8, 69: 8, 70: 8, 71: 8, 72: 8, 73: 0, 74: 10, 75: 0,
            76: 0, 77: 0, 78: 0, 79: 8, 80: 8, 81: 8, 82: 8, 83: 8,
        }
        self.matrix['platform_id'] = self.matrix['item_category_id_x'].map(platform_map)
        
        supercat_map = {
            0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 2, 9: 2, 10: 1, 11: 1, 12: 1,
            13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 3, 19: 3, 20: 3, 21: 3, 22: 3, 23: 3,
            24: 3, 25: 0, 26: 2, 27: 3, 28: 3, 29: 3, 30: 3, 31: 3, 32: 2, 33: 2, 34: 2,
            35: 2, 36: 2, 37: 4, 38: 4, 39: 4, 40: 4, 41: 4, 42: 5, 43: 5, 44: 5, 45: 5,
            46: 5, 47: 5, 48: 5, 49: 5, 50: 5, 51: 5, 52: 5, 53: 5, 54: 5, 55: 6, 56: 6,
            57: 6, 58: 6, 59: 6, 60: 6, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0,
            68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 7, 74: 7, 75: 7, 76: 7, 77: 7, 78: 7,
            79: 2, 80: 2, 81: 0, 82: 0, 83: 0
        }
        self.matrix['supercategory_id'] = self.matrix['item_category_id_x'].map(supercat_map)

    
    def add_item_name_groups(self, items, sim_thresh, feature_name="item_name_group"):
        import re
        from fuzzywuzzy import fuzz
        def partialmatchgroups(items, sim_thresh=sim_thresh):
            def strip_brackets(string):
                string = re.sub(r"\(.*?\)", "", string)
                string = re.sub(r"\[.*?\]", "", string)
                return string
    
            items = items.copy()
            items["nc"] = items.item_name.apply(strip_brackets)
            items["ncnext"] = np.concatenate((items["nc"].to_numpy()[1:], np.array([""])))
    
            def partialcompare(s):
                return fuzz.partial_ratio(s["nc"], s["ncnext"])
    
            items["partialmatch"] = items.apply(partialcompare, axis=1)
            # Assign groups
            grp = 0
            for i in range(items.shape[0]):
                items.loc[i, "partialmatchgroup"] = grp
                if items.loc[i, "partialmatch"] < sim_thresh:
                    grp += 1
            items = items.drop(columns=["nc", "ncnext", "partialmatch"])
            return items
    
        items = partialmatchgroups(items)
        items = items.rename(columns={"partialmatchgroup": feature_name})
        items = items.drop(columns="partialmatchgroup", errors="ignore")
    
        items[feature_name] = items[feature_name].apply(str)
        items[feature_name] = items[feature_name].factorize()[0]
        self.matrix = self.matrix.merge(items[["item_id", feature_name]], on="item_id", how="left")
    
    #mb add float and etc
    def reduce_memory_usage(self):
        numeric_cols = self.matrix.select_dtypes(include=['float64', 'int64'])
        for i in numeric_cols:
            max_value = self.matrix[i].max()
            if max_value < 127:
                self.matrix[i] = self.matrix[i].astype(np.int8)
            elif max_value < 32767:
                self.matrix[i] = self.matrix[i].astype(np.int16)
            elif max_value < 2147483647:
                self.matrix[i] = self.matrix[i].astype(np.int32)
            else:
                self.matrix[i] = self.matrix[i].astype(np.int64)
            
            
    def sort_data(self, column_name):
        self.matrix.sort_values(by=column_name, inplace=True)
    
    def get_data(self):
        return self.matrix

In [10]:
FE = Feature_extraction(train, test)

FE.preprocess()
FE.add_revenue(cols=["date_block_num", "shop_id", "item_id"])
FE.reduce_test_memory()
FE.concatenate_to_mx(cols=["date_block_num", "shop_id", "item_id"])
FE.lag_feature([1,2,3,6,12], ["item_cnt_month"])
FE.sort_data('date_block_num')
FE.add_shop_age()
FE.add_avg_sales()
FE.add_global_item_age()
FE.sort_data('date_block_num')
print('additional features')
shops=pd.read_csv("/home/siarhei/Programming/ML/Data/Predict Future Sales/shops.csv")
FE.add_city_codes(shops = shops)
FE.add_shop_type(shops = shops)
del shops
items=pd.read_csv("/home/siarhei/Programming/ML/Data/Predict Future Sales/items.csv")
FE.add_cats(items=items)
FE.add_item_name_groups(items=items, sim_thresh=70)

item_cnt_month
additional features


In [11]:
FE.reduce_memory_usage()
del items
del train
del test
data = FE.get_data()
del FE

In [12]:
import sys
sys.getsizeof(data)/1048576

817.9179601669312

In [13]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

class ModelValidator:
    def __init__(self, train, dev, target_name):
        self.train_x = train.drop([target_name], axis=1) 
        self.train_y = train[target_name]
        
        self.dev_x = dev.drop([target_name], axis=1)
        self.dev_y = dev[target_name]
        
        self.model = None

    def get_model(self):
        return self.model
    
    def get_dev(self):
        return self.dev_x, self.dev_y
    
    def get_train(self):
        return self.train_x, self.train_y

    
    def set_model(self, model_name):
        if model_name == 'xgbr':
            import gc
            import pickle
            from xgboost import XGBRegressor
            from matplotlib.pylab import rcParams
            self.model = XGBRegressor(max_depth=4,n_estimators=10)
    
    def train_model(self):
        self.model.fit(self.train_x,self.train_y,eval_metric="rmse")
        y_pred = self.model.predict(self.dev_x)
        print(f'\nMSE : {mean_squared_error(self.dev_y, y_pred)}\nMAE : {mean_absolute_error(self.dev_y, y_pred)}\n R2 : {r2_score(self.dev_y, y_pred)}')

In [14]:
data.columns
data.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)

In [15]:
train = data[data.date_block_num<=30]
test = data[(data.date_block_num>30)&(data.date_block_num<=33)]
kaggle = data[data.date_block_num==34].drop(['item_cnt_month'], axis=1)
del data
mv = ModelValidator(train, test, target_name='item_cnt_month')
mv.set_model(model_name='xgbr')
mv.train_model()
model = mv.get_model()
del mv

Y_test = model.predict(kaggle)
test = pd.read_csv('/home/siarhei/Programming/ML/Data/Predict Future Sales/test.csv')
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('xgb_submission.csv', index=False)


MSE : 22.60926055908203
MAE : 0.377547025680542
 R2 : 1.0


The resulting prediction gave me Score: 1.8718 on Kaggle

Of course this is definitely not a good score

but it shows that the data is processed correctly and the necessary structure is saved for Submission